In [ ]:
import random
import numpy as np
from utils.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Pour automatiquement recharger les modules externes
# voir http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Charger la banque de données CIFAR-10, prétraiter les images et ajouter une dimension pour le biais.
    
    Input :
    - num_training : nombre d'images à mettre dans l'ensemble d'entrainement
    - num_validation : nombre d'images à mettre dans l'ensemble de validation
    - num_test : nombre d'images à mettre dans l'ensemble de test
    - num_dev : d'images à mettre dans l'ensemble dev
    
    Output :
    - X_train, y_train : données et cibles d'entrainement
    - X_val, y_val: données et cibles de validation
    - X_test y_test: données et cibles de test 
    - X_dev, y_dev: données et cicles dev
    """
    # Charger les données CIFAR-10
    cifar10_dir = 'datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
  
    # Séparer en ensembles d'entraînement, de validation, de test et de dev
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    X_train = X_train.transpose(0, 3, 1, 2)
    X_test = X_test.transpose(0, 3, 1, 2)
    X_val = X_val.transpose(0, 3, 1, 2)
    X_dev = X_dev.transpose(0, 3, 1, 2)

    # Normalisation
    X_train -= np.mean(X_train, axis = 0)
    X_val -= np.mean(X_val, axis = 0)
    X_test -= np.mean(X_test, axis = 0)
    X_dev -= np.mean(X_dev, axis = 0)

    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev

In [ ]:
from layers.BatchNorm import SpatialBatchNorm
from layers.Conv import Conv2DCython
from layers.Dense import Dense
from layers.Flatten import Flatten
from layers.MaxPool import MaxPool2DCython
from model.Model import Model
from utils.model_loss import cross_entropy_loss_npdl

input_size = 32

# paramètres de convolution
num_filters = 32
filter_size = 7
channels = 3
stride = 1
pad = int((filter_size - 1)/2)

# paramètres de maxpooling
conv_out_size = input_size
pool_out_size = int(conv_out_size / 2)

# paramètres dense
num_classes = 10

def create_model(hidden_dim, weight_scale):
    model = Model()
    
    conv1 = Conv2DCython(num_filters, filter_size=filter_size, channels=channels, padding=pad, weight_scale=weight_scale)
    batchnorm1 = SpatialBatchNorm(num_filters, activation='relu')
    maxpool1 = MaxPool2DCython(pooling_size=2, stride=2)
    
    flatten1 = Flatten()
    dense1 = Dense(dim_input=num_filters*pool_out_size*pool_out_size, dim_output=hidden_dim, weight_scale=weight_scale, activation='relu')
    dense2 = Dense(dim_input=hidden_dim, dim_output=num_classes, weight_scale=weight_scale)
    
    model.add(conv1)
    model.add(batchnorm1)
    model.add(maxpool1)
    model.add(flatten1)
    model.add(dense1)
    model.add(dense2)
    model.add_loss(cross_entropy_loss_npdl)
    
    return model


def create_fully_conv_cnn(hidden_dim, weight_scale):
    model = Model()
    
    conv1 = Conv2DCython(num_filters, filter_size=filter_size, channels=channels, padding=pad, weight_scale=weight_scale)
    batchnorm1 = SpatialBatchNorm(num_filters, activation='relu')
    maxpool1 = MaxPool2DCython(pooling_size=2, stride=2)
    
    conv2 = Conv2DCython(hidden_size, filter_size=pool_out_size, channels=num_filters, weight_scale=weight_scale, activation='relu')
    conv3 = Conv2DCython(num_classes, filter_size=1, channels=hidden_size, weight_scale=weight_scale)
    flatten = Flatten()
    
    model.add(conv1)
    model.add(batchnorm1)
    model.add(maxpool1)
    model.add(conv2)
    model.add(conv3)
    model.add(flatten)
    model.add_loss(cross_entropy_loss_npdl)
    
    return model

In [ ]:
from model.Solver import Adam


model = create_model(100, 1e-2)
optimizer = Adam(1e-3, model)

In [ ]:
from model.Experimenter import Experimenter

experimenter = Experimenter(optimizer)

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

In [ ]:
data = {
    'X':X_train[:1000],
    'y':y_train[:1000],
}

hyperparams_grid = {
    'lr':[1e-2, 1e-3],
    'reg':[0.0, 0.1],
    'batch_size':[200],
    'lr_decay':[1.0, 0.95]
}

hyperparams_random = {
    'lr':(1e-2, 1e-3),
    'reg':(0.0, 0.1),
    'lr_decay':(1.0, 0.95),
    'batch_size':200
}

In [ ]:
best_hyperparams_grid, best_experiment_id_grid = experimenter.hyperparam_grid_search(data, epochs=2, K=2, hyperparams=hyperparams_grid, experiment_id='grid')

In [ ]:
experimenter.visualize_experiment_results(best_experiment_id_grid)

In [ ]:
best_hyperparams_rdm, best_experiment_id_rdm = experimenter.hyperparam_random_search(data, epochs=2, K=2, vals_per_hyperparam=2, hyperparams=hyperparams_random, experiment_id='random')

In [ ]:
experimenter.visualize_experiment_results(best_experiment_id_rdm)